<a href="https://colab.research.google.com/github/Jacob-Francis/SWSG_repo/blob/main/Colab_nb_hugo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Jacob-Francis/SWSG_repo.git

Cloning into 'SWSG_repo'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 3), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), done.
Resolving deltas: 100% (3/3), done.


In [2]:
%cd SWSG_repo/

/content/SWSG_repo


In [16]:
#!apt-get install cuda=10.2.89-1
%matplotlib inline

!pip install pykeops[colab] > install.log
!pip install geomloss[full]
!apt-get install ffmpeg

import torch
import numpy as np
from geomloss import SamplesLoss
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Device configuration
#pytorch.cuda.alloc.conf = expandable_segments
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
torch.set_default_device(device)

  Preparing metadata (setup.py) ... done
  Created wheel for geomloss: filename=geomloss-0.2.6-py3-none-any.whl size=32246 sha256=0b5d963486aa6be8990feb15f40f8e0b47874a337a9b3c81e2b1b1133807309d
  Stored in directory: /root/.cache/pip/wheels/0d/c9/80/4387eb03aa215ae557869d6fe8be498fd3d3cf297db2357b67
Successfully built geomloss
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [17]:
def Lloyds(f, N, k=10, Nsteps=3,scali=0.99, epsilon=0.001):

    # Generate initial grid of points for x
    grid_size = int(N**0.5)  # Assuming N is a perfect square
    x = (torch.rand(N, 2,device=device) - 0.5).detach().requires_grad_(True) # Random points in [-1, 1]
    # Generate fine grid of points y with k*N points
    fine_grid_size = int((k * N)**0.5)
    y = torch.stack(torch.meshgrid(torch.linspace(-0.5, 0.5, fine_grid_size,device=device),
                                   torch.linspace(-0.5, 0.5, fine_grid_size,device=device)), dim=-1).reshape(-1, 2)
    y = y[:k * N]

    # Compute weights b proportional to f(y)
    b = f(y)
    b /= b.sum()  # Normalize to make b a probability distribution

    # Uniform weights for points x
    a = torch.full((N,), 1 / N,device=device)

    # Define the entropic OT loss function
    loss_fn = SamplesLoss("sinkhorn", p=2,scaling=scali, blur=epsilon**0.5)

    # Iterative Lloyd's algorithm
    for _ in range(Nsteps):
        # Compute the transport map between x and (y, b)
        loss = loss_fn(a,x,b, y)
        grad = torch.autograd.grad(loss, x)[0]
        x=x.clone().detach().requires_grad_(True)

        # Move points in the direction of the gradient to minimize OT cost
        with torch.no_grad():
            x -= N * grad  # Adjust x by moving in gradient direction

    return x.detach()  # Return points that fit the density function

def init_gaussian(N, k=10, epsilon=0.001,Nsteps=3,scali=0.99,std=[0.2,0.2]):
    def gaussian_density(points, mean=[0, 0], std=[0.1, 0.1]):
        """Evaluates a Gaussian function at given points."""
        dist = torch.distributions.MultivariateNormal(
            torch.tensor(mean,device=device),
            torch.diag(torch.tensor(std,device=device))**2
        )
        return torch.exp(dist.log_prob(points))
    # Run Lloyd's algorithm
    f = lambda y: gaussian_density(y, mean=[0, 0], std=[0.2, 0.2])  # Gaussian density function
    points = Lloyds(f, N, k=k, Nsteps=Nsteps, epsilon=epsilon)
    return points

In [18]:
N = 50
M = 10*N ; N,M = int(N**0.5)**2,int(M**0.5)**2
debiase,scali,epsilon =False,0.99,0.001

#initialisations
grid_size = int(M ** 0.5)
grid = torch.linspace(-.5,.5, grid_size)
y1, y2 = torch.meshgrid(grid, grid, indexing='ij')
Y = torch.stack([y1.flatten(), y2.flatten()], dim=1)[:M]

G=init_gaussian(N,std=[0.3,0.3])


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [32]:
def sinkhorn_3marginals(G, Y, h_j, sigma, epsilon, num_iter=50, plot_every=5):
    N, D = G.shape
    M = Y.shape[0]
    X = G.clone()
    f_i, f_j, f_k = torch.zeros(N), torch.zeros(M), torch.zeros(N)
    one_over_sigma = 1.0 / sigma
    denom = 1 + one_over_sigma
    X_history = [X.detach().cpu().numpy()]
    f_k_history = [f_k.detach().cpu().numpy()]

    for t in range(num_iter):
        D_GX = torch.cdist(G, X)**2
        D_XY = torch.cdist(X, Y)**2
        C_total = D_GX[:, None, :] + one_over_sigma * D_XY.T[None, :, :]
        exponent = (f_i[:, None, None] + f_j[None, :, None] + f_k[None, None, :] - C_total) / epsilon
        max_exponent = exponent.max()
        gamma = torch.exp(exponent - max_exponent)
        gamma /= gamma.sum()

        f_i = -epsilon * torch.logsumexp(exponent, dim=(1, 2))
        f_j = -epsilon * torch.logsumexp(exponent, dim=(0, 2))
        f_k = -epsilon * torch.logsumexp(exponent, dim=(0, 1))

        Numerator_G = torch.einsum('imk,id->kd', gamma, G)
        Numerator_Y = torch.einsum('imk,jd->kd', gamma, Y)
        X = (Numerator_G + one_over_sigma * Numerator_Y) / denom

        if t % plot_every == 0 or t == num_iter - 1:
            X_history.append(X.detach().cpu().numpy())
            f_k_history.append(f_k.detach().cpu().numpy())

    return X, X_history, f_k_history


In [ ]:

# Parameters
sigma, epsilon, num_iter, plot_every = 1.0, 0.01, 5, 1

# Generate data
#G = torch.randn(N, D)
h_j = torch.full((M,), 1.0 / M)

# Run the algorithm
X_updated, X_history,f_k_history = sinkhorn_3marginals(G, Y, h_j, sigma, epsilon, num_iter, plot_every)

# Plotting the evolution of X_k
G_np = G.cpu().numpy()
Y_np = Y.cpu().numpy()

for idx, X_k_np in enumerate(X_history):
    iteration = idx * plot_every
    plt.figure(figsize=(8, 6))
    plt.scatter(G_np[:, 0], G_np[:, 1], color='blue', label='G_i', alpha=0.7)
    plt.scatter(Y_np[:, 0], Y_np[:, 1], color='green', s=10, label='Y_j', alpha=0.5)
    plt.scatter(X_k_np[:, 0], X_k_np[:, 1], color='red', label=f'X_k (Iter {iteration})', marker='x', s=80)
    plt.legend()
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title(f'Iteration {iteration}')
    plt.show()

for idx, f_k_np in enumerate(f_k_history):
    iteration = idx * plot_every
    plt.figure(figsize=(8, 6))
    plt.plot(f_k_np, label=f'f_k (Iter {iteration})')
    plt.legend()
    plt.xlabel('Iteration')
    plt.ylabel('f_k')
